In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.ensemble import ExtraTreesClassifier
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_columns',None)
df = pd.read_csv("melbourne_housing (1).csv")
df1 = df.copy()
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,03-09-2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,03-12-2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,04-02-2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,04-02-2016,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,04-03-2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# Dropping YearBuilt & BildinArea column as having more than 50% null values
# Dropping Postcode
df = df.drop(['YearBuilt','Postcode','BuildingArea'],axis=1)

-

## EDA and Preprocessing

-

In [ ]:
plt.figure(figsize=(12,5))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

#### Ploting count columns having less than 15 unique values

In [3]:
cols1 = [col for col in df.columns if len(df[col].unique()) < 15]
len(cols1)

5

#### Plotting Continuous Columns

In [4]:
cols2 = [col for col in df.columns if df[col].dtypes in ['int64','float64']]
len(cols2)

13

##### Joint plot with respect to all numerical features

##### Lookign for outliers

In [5]:
cols3 = [col for col in df.columns if df[col].dtypes in ['float64'] and col not in ['Car','Rooms','Bedroom2','Bathroom']]
print(len(cols2))
cols3

13


['Price',
 'Distance',
 'Postcode',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [6]:
## Function to detect outliers

def find_outliers(col):
    
    outliers = []
    
    data_std = np.std(df[col])
    data_mean = np.mean(df[col])
    
    lower_limit = data_std - (data_std * 3)
    upper_limit = data_std + (data_std *3)
    
    outliers = df[(df[col] < lower_limit) & (df[col] > upper_limit)]
    
    return len(outliers)

In [7]:
for col in cols3:
    print(f'{col} : {find_outliers(col)}')

Price : 0
Distance : 0
Postcode : 0
Landsize : 0
BuildingArea : 0
YearBuilt : 0
Lattitude : 0
Longtitude : 0
Propertycount : 0


In [ ]:
## Checking Median and Mode of remaining columns
def Stat(col):
    print(f'Mean : {df[col].mean()}')
    print(f'Median : {df[col].median()}')
    print(f'Mode : {df[col].mode()[0]}')

#### Distribution of price before Transformation

In [ ]:
plt.figure(figsize=(12,6))
print('Min : {}   Max : {}'.format(min(df1['Price'].dropna()),max(df1['Price'].dropna())))
plt.xlim(85000,4000000)
sns.distplot(df['Price'].dropna(),bins=150)

In [ ]:
import scipy.stats as stat
import pylab 
import statsmodels.api as sm

In [ ]:
#### If you want to check whether feature is guassian or normal distributed
#### Q-Q plot
def plot_data(df,feature):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    df[feature].hist()
    #plt.subplot(1,2,2)
    #stat.probplot(df[feature],dist='norm',plot=pylab)
    #plt.show()
    sm.qqplot(df[feature],line='q')
    pylab.show()

In [ ]:
plot_data(df,'Price')

#### Logrithmic Transformation

In [ ]:
df1['Price Log'] = np.log(df1['Price'])
plot_data(df1,'Price Log')

#### Expotential Tarnsformation

In [ ]:
df1['Price Expo']  =  df1['Price'] ** (1/1.2)
plot_data(df1,'Price Expo')

In [8]:
### Transforming Price Using Logrithmic Transformation
df['Price'] = np.log(df['Price'])

In [ ]:
plot_data(df,'Landsize')

In [ ]:
plot_data(df,'Lattitude')

In [ ]:
plot_data(df,'Longtitude')

In [ ]:
plt.figure(figsize=(12,6))
print('Min : {}   Max : {}'.format(min(df['Price'].dropna()),max(df['Price'].dropna())))
#plt.xlim(85000,4000000)
sns.distplot(df['Price'].dropna(),bins=100)

In [ ]:
plt.figure(figsize=(12,6))
print('Min : {}   Max : {}'.format(min(df['Price'].fillna(df['Price'].mean())),max(df['Price'].fillna(df['Price'].mean()))))
#plt.xlim(85000,4000000)
plt.ylim(0,2)
sns.distplot(df['Price'].fillna(df['Price'].mean()),bins=150)

In [ ]:
plt.figure(figsize=(12,6))
#print('Min : {}   Max : {}'.format(min(df['Price'].fillna(df['Price'].mean())),max(df['Price'].fillna(df['Price'].mean()))))
#plt.xlim(85000,4000000)
plt.ylim(0,2)
sns.distplot(df['Price'].fillna(df['Price'].median()),bins=150)

In [ ]:
plt.figure(figsize=(12,6))
#print('Min : {}   Max : {}'.format(min(df['Price'].fillna(df['Price'].mean())),max(df['Price'].fillna(df['Price'].mean()))))
#plt.xlim(85000,4000000)
plt.ylim(0,2)
sns.distplot(df['Price'].fillna(df['Price'].mode()[0]),bins=150)

### Handling Missing values

##### Filling the null values with Mean/Median/Mode

In [ ]:
df.isnull().sum()

In [9]:
## Filling Bedroom,Bathroom and car with Mode

for col in ['Bedroom2','Bathroom','Car']:
    df[col] = df[col].fillna(df[col].mode()[0])

In [ ]:
### Filling Null values with Median

In [10]:
def Fill_median(col):
    df[col].fillna(df[col].median(),inplace=True)

In [11]:
for col in ['Price','Landsize','Lattitude','Longtitude']:
    Fill_median(col)

In [12]:
df.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2             0
Bathroom             0
Car                  0
Landsize             0
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude            0
Longtitude           0
Regionname           3
Propertycount        3
dtype: int64

In [13]:
### Droppping Remaining Rows
df.dropna(inplace=True)

In [ ]:
# Plotting Chart to see null values

plt.figure(figsize=(12,5))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

### Feature Encoding

In [ ]:
df.head()

In [ ]:
print('Count of unique values in all features : ')
df.nunique()

In [14]:
## Dropping Address , Suburb , 'SellerG'  due to high cardinality
df = df.drop(['Address','Suburb','SellerG'],axis=1)

In [15]:
### List of categorical features having less than 50 uniqe values

lst = [col for col in df.columns if df[col].dtypes == 'object' and len(df[col].unique()) < 50]
lst

['Type', 'Method', 'CouncilArea', 'Regionname']

In [16]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [17]:
for col in lst:
    df[col] = LE.fit_transform(df[col])

In [ ]:
df.head()

#### Feature Scaling

In [18]:
from sklearn.preprocessing import StandardScaler
Std = StandardScaler()

In [19]:
df2 = pd.DataFrame(Std.fit_transform(df.drop('Date',axis=1)),columns=df.drop('Date',axis=1).columns)

In [ ]:
df2.head()

### Feature Selection

In [20]:
X = df2.drop('Price',axis=1)
Y = df2['Price']

In [21]:
from sklearn.feature_selection import SelectKBest,chi2,f_classif

In [22]:
X.dropna(inplace=True)

In [23]:
Feat = SelectKBest(score_func=f_classif,k=10)
BestFeat = Feat.fit(X,Y)

cols = pd.DataFrame(X.columns)
score = pd.DataFrame(BestFeat.scores_)
BF = pd.concat([cols,score],axis=1)
BF.columns = ['Feature','Score']
BF

,Feature,Score
0,Rooms,2.813637
1,Type,2.737971
2,Method,0.805016
3,Distance,1.476564
4,Postcode,0.962008
5,Bedroom2,2.677000
6,Bathroom,2.310036
7,Car,1.196352
8,Landsize,0.691693
9,BuildingArea,3.465943


In [24]:
BestFeatures = pd.DataFrame(BF.nlargest(12,'Score'))
BestFeatures

,Feature,Score
9,BuildingArea,3.465943
0,Rooms,2.813637
1,Type,2.737971
5,Bedroom2,2.677000
6,Bathroom,2.310036
10,YearBuilt,1.857076
13,Longtitude,1.508461
3,Distance,1.476564
12,Lattitude,1.444948
7,Car,1.196352


### Model Fitting

In [25]:
X = X[BestFeatures['Feature']]

In [26]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [27]:
## Linear Regression
from sklearn.linear_model import LinearRegression
Reg_model = LinearRegression()

In [28]:
Reg_model.fit(X_train,Y_train)
Y_pred = Reg_model.predict(X_test)

In [29]:
from sklearn.metrics import *

In [30]:
r2 = r2_score(Y_test,Y_pred)
print("R2 score", r2)
print(Reg_model.score(X_train,Y_train))
print(Reg_model.score(X_test,Y_test))

R2 score 0.5298320766475566
0.5306734912198712
0.5298320766475566


In [31]:
### Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from memory_profiler import profile

In [32]:
X1_train,X1_test,Y1_train,Y1_test = train_test_split(X,Y,test_size=0.25,random_state=0)
RF_model = RandomForestRegressor(n_jobs=-1)

In [33]:
RF_model.fit(X1_train,Y1_train)
Y1_pred = RF_model.predict(X1_test)

In [34]:
from sklearn.metrics import accuracy_score
#accuracy_score(Y_test,Y_pred)
RF_model.score(X1_test,Y1_test)

0.5917333016706261

In [35]:
## KNN
from sklearn.neighbors import KNeighborsRegressor

KNN_model = KNeighborsRegressor(12)

In [36]:
KNN_model.fit(X_train,Y_train)

KNeighborsRegressor(n_neighbors=12)

In [37]:
KNN_model.score(X_test,Y_test)

0.5728870850850591

## Hyper Parameter Tuning

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [42]:
RF_params = {
                'criterion' : ['mse','mae'],
                'min_samples_leaf' : [1,2,3],
                'max_leaf_nodes' : [2,3,4],
                'random_state' : [1,5,42],
                'ccp_alpha' : [0.0,0.2,0.5,1,10],
             }

In [46]:
KNN_params = {
                'n_neighbors' : [5,7,10],
                'weights' : ['uniform','distance'],
                'algorithm' : ['auto','ball_tree','kd_tree','brute'],
                'p' : [2,3,5],
             }

In [47]:
RandmSV = RandomizedSearchCV(
                        KNN_model,
                        param_distributions=KNN_params,
                        cv=3,
                        n_jobs=-1
                     )

RandmSV.fit(X_train,Y_train)
result = pd.DataFrame(RandmSV.cv_results_)

In [48]:
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_weights,param_p,param_n_neighbors,param_algorithm,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.070576,0.004567,0.509800,0.015200,distance,2,10,auto,"{'weights': 'distance', 'p': 2, 'n_neighbors':...",0.556597,0.544853,0.539815,0.547088,0.007031,1
1,0.134165,0.104171,6.458012,0.141090,uniform,3,10,auto,"{'weights': 'uniform', 'p': 3, 'n_neighbors': ...",0.554066,0.536455,0.529149,0.539890,0.010458,3
2,0.007203,0.000016,27.880806,0.131784,distance,5,5,brute,"{'weights': 'distance', 'p': 5, 'n_neighbors':...",0.517731,0.498458,0.496647,0.504278,0.009541,10
3,0.082475,0.018048,6.018718,0.048715,uniform,5,10,kd_tree,"{'weights': 'uniform', 'p': 5, 'n_neighbors': ...",0.544202,0.527717,0.525328,0.532416,0.008391,4
4,0.127037,0.003025,25.463494,0.300183,distance,3,7,ball_tree,"{'weights': 'distance', 'p': 3, 'n_neighbors':...",0.537821,0.518350,0.518876,0.525016,0.009057,6
5,0.080969,0.011589,4.907798,0.097807,distance,3,5,kd_tree,"{'weights': 'distance', 'p': 3, 'n_neighbors':...",0.524338,0.499488,0.499906,0.507911,0.011617,9
6,0.007904,0.000484,27.818804,0.038938,uniform,3,7,brute,"{'weights': 'uniform', 'p': 3, 'n_neighbors': ...",0.538607,0.518931,0.516840,0.524793,0.009806,8
7,0.083656,0.006826,6.435622,0.123174,distance,3,10,kd_tree,"{'weights': 'distance', 'p': 3, 'n_neighbors':...",0.553719,0.536411,0.532726,0.540952,0.009152,2
8,0.008167,0.000827,1.851226,0.075939,distance,2,7,brute,"{'weights': 'distance', 'p': 2, 'n_neighbors':...",0.542649,0.523858,0.523099,0.529869,0.009043,5
9,0.072459,0.017584,5.444737,0.178748,uniform,3,7,auto,"{'weights': 'uniform', 'p': 3, 'n_neighbors': ...",0.538580,0.518939,0.516911,0.524810,0.009772,7


In [49]:
print(f'KNN Best Parameters :\n\n {RandmSV.best_params_}\n\n')
print(f'KNN Best Score :\n\n {RandmSV.best_score_}')

KNN Best Parameters :

 {'weights': 'distance', 'p': 2, 'n_neighbors': 10, 'algorithm': 'auto'}


KNN Best Score :

 0.5470883879108378
